## Load data and remove rows from crowdsoucing with less than 3 totalVotes

In [2]:
import pandas as pd
crowdsourcing = pd.read_csv( './anonymized_Crowdsourced_data.csv')
#Drop the columns that label value is none
crowdsourcing = crowdsourcing.dropna(subset=['initialLabel'])
print(len(crowdsourcing))
crowdsourcing.sample(2)


7051


/Users/yunong/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,initialLabel,binaryInitialLabel,title,url,trueVoterUniqueIDs,falseVoterUniqueIDs,numTrueVotes,numFalseVotes,totalVotes,maxVotes,interraterPercentAgreement,agreementWithLabel,votersLabel,voterLabelWasAccurate,averageInterraterPercentAgreement,averageAgreementWithLabel,percentOfAgreeingDecisions
2349,2349,FALSE,0.0,Multiple posts shared repeatedly on Facebook ...,https://www.poynter.org/?ifcn_misinformation=m...,['User291'],"['User79', 'User678', 'User4']",1.0,3.0,4.0,3.0,0.75,0.75,0.0,1.0,NaN,NaN,NaN
4879,4879,TRUE,1.0,Zydus Cadila's COVID-19 vaccine candidate foun...,https://www.reuters.com/article/us-health-coro...,"['User536', 'User452', 'User314']",['User708'],3.0,1.0,4.0,3.0,0.75,0.75,1.0,1.0,NaN,NaN,NaN


In [3]:
#remove rows from crowdsoucing with less than 2 totalVotes
crowdsourcing = crowdsourcing[crowdsourcing['totalVotes'] > 2]
len(crowdsourcing)

6931

In [4]:
import numpy as np
#create a matrix using numTrueVotes as the first column and numFalseVotes as the second column
m= np.array([crowdsourcing['numTrueVotes'], crowdsourcing['numFalseVotes']]).T
m.shape

(6931, 2)

Krippendorff's alpha
Github from: https://github.com/grrrr/krippendorff-alpha

In [5]:
#! /usr/bin/env python
# -*- coding: utf-8
'''
Python implementation of Krippendorff's alpha -- inter-rater reliability

(c)2011-17 Thomas Grill (http://grrrr.org)

Python version >= 2.4 required
'''

from __future__ import print_function
try:
    import numpy as np
except ImportError:
    np = None


def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2


def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or 
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.


if __name__ == '__main__': 
    print("Example from http://en.wikipedia.org/wiki/Krippendorff's_Alpha")

    data = (
        "*    *    *    *    *    3    4    1    2    1    1    3    3    *    3", # coder A
        "1    *    2    1    3    3    4    3    *    *    *    *    *    *    *", # coder B
        "*    *    2    1    3    4    4    *    2    1    1    3    3    *    4", # coder C
    )

    missing = '*' # indicator for missing values
    array = [d.split() for d in data]  # convert to 2D list of string items
    
    print("nominal metric: %.3f" % krippendorff_alpha(array, nominal_metric, missing_items=missing))
    print("interval metric: %.3f" % krippendorff_alpha(array, interval_metric, missing_items=missing))

Example from http://en.wikipedia.org/wiki/Krippendorff's_Alpha
nominal metric: 0.691
interval metric: 0.811


In [6]:
#append all usernames appearing in the list in trueVoterUniqueIDs to the username list in the dataframe
usernames = []
for l in crowdsourcing['trueVoterUniqueIDs']:
    for i in l[1:-1].replace(' ','').replace('\'','').split(','):
        if i not in usernames and i != '':
            usernames.append(i)

for l in crowdsourcing['falseVoterUniqueIDs']:
    for i in l[1:-1].replace(' ','').replace('\'','').split(','):
        if i not in usernames and i != '':  
            usernames.append(i)

In [7]:
#for each row in the dataframe, if user_name in trueVoterUniqueIDs, append 1 to data_str, else if user_name in falseVoterUniqueIDs append 0, else append '*'
data = ()
for user_name in usernames:
    data_str = ''
    for i in range(len(crowdsourcing)):
        if user_name in crowdsourcing.iloc[i]['trueVoterUniqueIDs']:
            data_str += '1'
        elif user_name in crowdsourcing.iloc[i]['falseVoterUniqueIDs']:
            data_str += '0'
        else:
            data_str += '*'
        data_str += ' '
    data += (data_str,)
    print(user_name)


User0
User1
User2
User3
User4
User5
User6
User7
User8
User9
User10
User11
User12
User13
User14
User15
User16
User17
User18
User19
User20
User21
User22
User23
User24
User25
User26
User27
User28
User29
User30
User31
User32
User33
User34
User35
User36
User37
User38
User39
User40
User41
User42
User43
User44
User45
User46
User47
User48
User49
User50
User51
User52
User53
User54
User55
User56
User57
User58
User59
User60
User61
User62
User63
User64
User65
User66
User67
User68
User69
User70
User71
User72
User73
User74
User75
User76
User77
User78
User79
User80
User81
User82
User83
User84
User85
User86
User87
User88
User89
User90
User91
User92
User93
User94
User95
User96
User97
User98
User99
User100
User101
User102
User103
User104
User105
User106
User107
User108
User109
User110
User111
User112
User113
User114
User115
User116
User117
User118
User119
User120
User121
User122
User123
User124
User125
User126
User127
User128
User129
User130
User131
User132
User133
User134
User135
User136
User137
User13

In [71]:
missing = '*' # indicator for missing values
array = [d.split() for d in data]  # convert to 2D list of string items

print("nominal metric: %.3f" % krippendorff_alpha(array, nominal_metric, missing_items=missing))

nominal metric: 0.428


In [9]:
missing = '*' # indicator for missing values
array = [d.split() for d in data]  # convert to 2D list of string items

print("interval metric: %.3f" % krippendorff_alpha(array, interval_metric, missing_items=missing))

interval metric: 0.428


In [10]:
len(usernames)

756